## PDF Query Using Langchain

In [ ]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.9/273.9 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 1.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('/content/Idioms-LLM.pdf')

In [ ]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [ ]:
raw_text

'file:///C:/Documents%20and%20Settings/Owner/My%20Documen...\n1 of 5 1/29/2008 10:02 AMThe American Heritage® Dictionary of Idioms \nChristine Ammer \nOTHER BOOKS BY CHRISTINE AMMER \nHave a Nice Day! ?\nNo Problem! A Dictionary of Clichés Fruitcakes & Couch Potatoes and Other\nDelicious Expressions It\'s Raining Cats and Dogs and Other Beastly Expressions \nSouthpaws and Sunday Punches and Other Sporting Expressions Fighting Words \nfrom War, Rebellion and Other Combative Capers Seeing Red or Tickled Pink: \nColor Terms in Everyday Language The HarperCollins Dictionary of Music Unsung: \nA History of Women in American Music The New A to Z of Women\'s Health \nThe inclusion of any word or phrase in this book is not an expression of the \nPublisher\'s opinion as to whether or not it is subject to proprietary rights. No \ndefinition in this book is to be regarded as affecting the validity of any trademark. \nAmerican Heritage® and the eagle logo are registered trademarks of Forbes Inc. \

In [ ]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 30,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

n = 50
chunk_size = len(texts) // n

texts = texts[:n*chunk_size]

In [ ]:
texts[20]

'hand. see under ABOVE SUSPICION. \nabsence \nhand. In addition to the idiom beginning with ABSENCE, also see \nCONSPICUOUS BY ITS ABSENCE. \nabsence makes the heart grow fonder \nSeparation intensifies love, as in After a year in another country she accepted his \nproposal, so I guess absence makes the heart grow fonder,  or, used ironically, The \nboss leaves earlier every day; oh well, absence makes the heart grow fonder.\nAlthough versions of this saying date from Roman times, it only became popular \nafter Thomas Haynes Bayly used it as the last line of a song in The Isle of Beauty \n(1850). The opposite sentiment is expressed by FAMILIARITY BREEDS \nCONTEMPT. \nabsent without leave \nAway without permission or explanation, as in Her daughter went to the mall but'

In [ ]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [ ]:
document_search = FAISS.from_texts(texts, embeddings)

In [ ]:
document_search


In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [ ]:
query = "absent without leave"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Away without permission or explanation, as in Her daughter went to the mall but got in trouble for being absent without leave.'

In [ ]:
def fetch_and_process_idioms(alphabet):
    """
    Fetches idioms for the given alphabet letter and processes the response to extract idioms.
    """
    query = f"Give me 5 idioms for the letter {alphabet}. I need response in this format: <Alphabet>: <list of idioms separated by comma>"
    docs = document_search.similarity_search(query)
    response = chain.run(input_documents=docs, question=query)

    # Assuming the response is a string "<Alphabet>: <Idiom 1>, <Idiom 2>, ..."
    # Here's how you might process that string to extract the idioms:
    processed_response = response.split(":")[1].strip() if ":" in response else ""
    idioms = processed_response.split(", ") if processed_response else []

    return idioms

def append_idioms_to_file(idioms_collection, filename):
    with open(filename, 'a+') as f:
        f.seek(0)
        existing_lines = f.read().splitlines()

        for idioms in idioms_collection.values():
            for idiom in idioms:
                if idiom not in existing_lines:
                    f.write(idiom + '\n')


def main():
    alphabets = [chr(i) for i in range(65, 91)]  # A-Z in uppercase
    idioms_collection = {}
    for alphabet in alphabets:
        idioms = fetch_and_process_idioms(alphabet)
        idioms_collection[f"{alphabet}"] = idioms

        append_idioms_to_file(idioms_collection, 'idioms.txt')
        if idioms:
            print(f"{alphabet}: {', '.join(idioms)}")
        else:
            print(f"No idioms found for {alphabet}")

for i in range(30):
  if __name__ == "__main__":
      main()


A: all the best, apple of one's eye, as easy as pie, at the drop of a hat, a dime a dozen
B: down in the dumps, down with the new group, busy as a beaver, where's the beef, beef up, bee in one's bonnet
C: call it a day, catch red-handed, cry over spilled milk, cut corners, chip on one's shoulder
D: dig up dirt, dilemma, daily dozen, drag in, daggers drawn
E: every cloud has a silver lining, easy as pie, elephant in the room, early bird catches the worm, eat humble pie
F: fly off the handle, face the music, feather in one's cap, fall off the wagon, fight tooth and nail
G: give the shirt off one's back, give the slip, give the time of day, give the word, go against the grain
H: hay hand, haystack hand, haywire hand, hazard hand, haze hand
I: in a nutshell, in the limelight, in the loop, in the red, in the same boat
J: jumping the gun, joined at the hip, jump ship, jump on the bandwagon, jumping for joy
K: keep an eye out, kick the bucket, kill two birds with one stone, keep your chin up,

KeyboardInterrupt: 

## **Idioms to Context Generation**

In [ ]:
import openpyxl
from openpyxl import Workbook

# Simulated function for fetching responses. Replace with actual API call.
def fetch_response(idiom):
    query = (f"Please provide a direct and succinct context for the idiom '{idiom}', "
             "followed by a concise sentence that uses the idiom. "
             "Format your answer as 'Context: [Brief scenario], Example: [Sentence using the idiom].'")
    docs = document_search.similarity_search(query)
    response = chain.run(input_documents=docs, question=query)
    return response

def process_response(response):
    # Split the response into context and example sentence
    parts = response.split(", Example: ")
    context = parts[0].replace("Context: ", "").strip()
    example_sentence = parts[1].strip() if len(parts) > 1 else ""
    return context, example_sentence

def main():
    idioms_file = "idioms.txt"
    wb = Workbook()
    ws = wb.active
    ws.append(["Idiom", "Context", "Example"])

    idioms_processed = 0
    save_interval = 5  # Save after every 5 responses

    with open(idioms_file, "r") as file:
        idioms = [line.strip() for line in file.readlines()]

    for idiom in idioms:
        response = fetch_response(idiom)
        context, example_sentence = process_response(response)
        ws.append([idiom, context, example_sentence])

        idioms_processed += 1
        if idioms_processed % save_interval == 0:
            wb.save("Idioms_Context.xlsx")
            print(f"Saved after {idioms_processed} idioms.")

    # Save any remaining idioms that didn't hit the save interval
    wb.save("Idioms_Context_Examples.xlsx")
    print("Final save completed.")

if __name__ == "__main__":
    main()


### Processing and Extracting Example

In [ ]:
import pandas as pd

# Load the dataset from an Excel file
file_name = '/content/Idioms_Context.xlsx'
df = pd.read_excel(file_name, engine='openpyxl')

# Specify the column you want to extract text from
column = 'Context'

# Extract text after "Example: " and place it under the existing example column
df['Example'] = df['Example'].str.strip('"')
df['Example'] = df[column].str.split('Example: ').str[-1]

# Save the DataFrame back to the same Excel file
df.to_excel(file_name, index=False)


In [ ]:
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator().from_loaders([loader])

In [ ]:
query = "Explain me about Attention is all you need"
index.query(query)

' Attention is All You Need is a paper published in 2017 by researchers from Google Brain. The paper introduces the Transformer, a model architecture that relies entirely on an attention mechanism to draw global dependencies between input and output, instead of using recurrence. The Transformer allows for significantly more parallelization and can reach a new state of the art in translation quality after being trained for as little as twelve hours on eight P100 GPUs. Additionally, self-attention could yield more interpretable models.'